# Firecrawl Tutorial: Web Scraping and Crawling for AI

Welcome to this tutorial on **Firecrawl**! 

Firecrawl is a powerful tool that simplifies web scraping and crawling, turning entire websites into clean, structured, LLM-ready data. It's designed to handle the complexities of modern web scraping, such as JavaScript rendering and anti-bot measures, allowing you to focus on data extraction and analysis.

In this notebook, we'll cover the essential concepts of Firecrawl, including:

* **Setup and Installation**
* **Scraping a single URL** with the `scrape` endpoint
* **Crawling an entire website** with the `crawl` endpoint

Let's get started! 🚀

## 1. Setup and Installation

First, we need to install the necessary Python libraries: `firecrawl-py` for interacting with the Firecrawl API and `python-dotenv` for managing our API key securely.

In [2]:
#!pip install firecrawl-py python-dotenv -q

### 1.1. API Key Management

To use Firecrawl, you'll need an API key. You can get one by signing up on the [Firecrawl website](https://www.firecrawl.dev/).

It's a best practice to store your API key in a `.env` file to keep it secure and out of your code. Create a file named `.env` in the same directory as this notebook and add your API key like this:

```
FIRECRAWL_API_KEY='your_api_key_here'
```

Now, let's load the API key from the `.env` file.

In [3]:
import os
from dotenv import load_dotenv
from firecrawl import FirecrawlApp

# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment variable
firecrawl_api_key = os.getenv("FIRECRAWL_API_KEY")

# Initialize the FirecrawlApp
app = FirecrawlApp(api_key=firecrawl_api_key)

## 2. Scraping a Single URL (`scrape`)

The `scrape` endpoint is perfect for extracting content from a single web page. It's fast, efficient, and returns the data in a clean format like Markdown.

Let's try scraping the Firecrawl documentation page.

In [5]:
# Define the URL to scrape
url_to_scrape = 'https://docs.firecrawl.dev/introduction'

# Scrape the URL
scraped_data = app.scrape_url(url_to_scrape)

# Print the scraped markdown content (supports SDKs that return a Pydantic model or a dict)
markdown_text = None
try:
    # Newer SDKs: ScrapeResponse model
    markdown_text = getattr(scraped_data, 'markdown', None)
except Exception:
    pass

if markdown_text is None:
    # Dict-like responses
    if isinstance(scraped_data, dict):
        markdown_text = scraped_data.get('markdown') or scraped_data.get('data', {}).get('markdown')
    # Pydantic v2 model fallback
    if markdown_text is None and hasattr(scraped_data, 'model_dump'):
        try:
            dumped = scraped_data.model_dump()
            markdown_text = dumped.get('markdown') or dumped.get('data', {}).get('markdown')
        except Exception:
            pass

print(markdown_text if markdown_text else 'No markdown content found')

[Firecrawl Docs home page![light logo](https://mintlify.s3.us-west-1.amazonaws.com/firecrawl/logo/logo.png)![dark logo](https://mintlify.s3.us-west-1.amazonaws.com/firecrawl/logo/logo-dark.png)](https://firecrawl.dev/)

v1

Search...

Ctrl KAsk AI

Search...

Navigation

Get Started

Quickstart

[Documentation](https://docs.firecrawl.dev/introduction) [SDKs](https://docs.firecrawl.dev/sdks/overview) [Learn](https://www.firecrawl.dev/blog/category/tutorials) [Integrations](https://www.firecrawl.dev/app) [API Reference](https://docs.firecrawl.dev/api-reference/introduction)

On this page

- [Welcome to Firecrawl](https://docs.firecrawl.dev/introduction#welcome-to-firecrawl)
- [How to use it?](https://docs.firecrawl.dev/introduction#how-to-use-it%3F)
- [API Key](https://docs.firecrawl.dev/introduction#api-key)
- [Features](https://docs.firecrawl.dev/introduction#features)
- [Powerful Capabilities](https://docs.firecrawl.dev/introduction#powerful-capabilities)
- [Installing Firecrawl](http

## 3. Crawling a Website (`crawl`)

The `crawl` endpoint is used to systematically browse and scrape an entire website. Firecrawl handles the complexities of discovering and navigating through the site's pages.

Let's crawl the Firecrawl blog. We'll set a `limit` to keep the crawl short for this example.

In [ ]:
# Define the URL to crawl
url_to_crawl = 'https://www.firecrawl.dev/blog'

# Use the SDK's ScrapeOptions model 
from firecrawl.firecrawl import ScrapeOptions

scrape_opts = ScrapeOptions(formats=['markdown'])

result = app.crawl_url(
    url_to_crawl,
    limit=2,
    scrape_options=scrape_opts,
    # You can also try:
    # max_depth=1,
    # ignore_sitemap=True,
)

# Normalize result into a list of items
items = None
if hasattr(result, 'data'):
    items = result.data
elif isinstance(result, dict):
    items = result.get('data') or result.get('results') or result
else:
    items = result  # already iterable in some SDKs

# Print crawled data robustly (handles object/dict and nested structures)
for item in items or []:
    # Resolve URL
    source_url = (
        getattr(item, 'source_url', None)
        or getattr(item, 'url', None)
        or (item.get('source_url') if isinstance(item, dict) else None)
        or (item.get('url') if isinstance(item, dict) else None)
    )
    # Resolve markdown
    markdown = getattr(item, 'markdown', None)
    if markdown is None and isinstance(item, dict):
        markdown = (
            item.get('markdown')
            or item.get('data', {}).get('markdown')
            or item.get('content', {}).get('markdown')
        )
    # Pydantic v2 fallback
    if markdown is None and hasattr(item, 'model_dump'):
        try:
            dumped = item.model_dump()
            markdown = (
                dumped.get('markdown')
                or dumped.get('data', {}).get('markdown')
                or dumped.get('content', {}).get('markdown')
            )
            source_url = source_url or dumped.get('source_url') or dumped.get('url')
        except Exception:
            pass

    print(f"URL: {source_url}")
    print(f"Markdown Content:\n{(markdown or '')[:1000]}\n---")

if not items:
    print('No items returned by crawl. Try removing www from the URL, increasing limit, or relaxing options.')

URL: None
Markdown Content:
Our biggest week of announcements is coming.Everything changes on August 19th.Everything changes on August 12th

[![How Dub Builds AI Affiliate Pages with Firecrawl](https://www.firecrawl.dev/customers/steven-tey-dub.webp)\\
\\
Aug 13, 2025\\
\\
**How Dub Builds AI Affiliate Pages with Firecrawl** \\
\\
Discover how Dub uses Firecrawl to power their AI page builder, transforming company websites into affiliate program landing pages in seconds.\\
\\
By Steven Tey](https://www.firecrawl.dev/blog/how-dub-builds-ai-affiliate-pages-with-firecrawl)

## Explore Articles

[All](https://www.firecrawl.dev/blog) [Product Updates](https://www.firecrawl.dev/blog/category/product) [Tutorials](https://www.firecrawl.dev/blog/category/tutorials) [Customer Stories](https://www.firecrawl.dev/blog/category/customer-stories) [Tips & Resources](https://www.firecrawl.dev/blog/category/tips-and-resources)

[![Web Scraping with n8n: 8 Powerful Workflow Templates](https://www.firecra

## 4. Advanced Features and Integrations

Firecrawl offers many advanced features and can be integrated with other tools to build powerful AI applications:

* **LLM Extraction:** Use natural language to extract specific data points from a page.
* **Integrations:** Firecrawl seamlesly integrates with popular AI frameworks and tools like LangChain, LangGraph, and Weaviate.
* **Asynchronous Operations:** Perform long-running crawl jobs without blocking your application.

For more in-depth information, be sure to check out the official [Firecrawl Documentation](https://docs.firecrawl.dev/).

## Conclusion

Congratulations! You've learned the basics of using Firecrawl to scrape and crawl websites. With these fundamental skills, you can now easily gather and process web data for your AI and data science projects. 🎉